# Sentinel-2 Datenverarbeitungs-Pipeline mit QGIS und Python

In den folgenden Schritten werden am Beispiel Uruguays mögliche Schritte der Datenverarbeitung von Sentinel-2-Daten aufgearbeitet und umgesetzt. Dabei werden Geoverarbeitungsschritte mit PyQGIS-Code eingebaut sowie verschiedene Methodikansätze getestet, die anschließend je nach Anwendungsfeld eingesetzt werden können. 

Es ist möglich, mittels Bau eines QGIS-Plugins direkt Daten vom Copernicus Dataspace zu beziehen. Dafür würde ein Authentifizierungsmodul mit Verwaltung der Zugangsdaten, automatischer Token-Erneuerung etc. benötigt werden, ebenso mit Qt Designer umsetzbare GUI-Elemente und spezifischen Code zur API-Integration und Umsetzung im GIS. Praktisch gibt es schon den "Sentinel-2 Image Downloader" von Murat Çalışkan zum kostenlosen Download als QGIS-Plugin.

## Sentinel-2 Daten herunterladen 

Wir nutzen nicht den manuellen Weg über den Copernicus Dataspace, sondern verwenden das erwähnte Sentinel-2-Downloadplugin. Als Datenvorbereitung habe ich die administrative level 0-2 boundaries von Uruguay als .shp heruntergeladen und in QGIS dissolved, um die Landfläche des Staates zu erhalten. 

In [ ]:
# Ansatz direkt in der QGIS Python-Konsole
params = {'INPUT':"C:/", # hier richtigen pfad einfuegen
          'FIELD':[],
          'SEPARATE_DISJOINT':False,
          'OUTPUT':'TEMPORARY_OUTPUT'}

processing.run("native:dissolve", params)

# Alternativ ohne Code als Geoverarbeitungswerkzeug in QGIS: Vector geometry --> Dissolve

# Ergebnis nach Belieben abspeichern

Im Plugin wählen wir als erstes bei "Download Footprints" unseren Layer aus und klicken auf "Layer Extent". Max. Cloud 10%, Product Type L1C (es gibt kein L2A für Uruguay), Start- und Enddatum den September 2025. Wir geben einen Outputfolder an.

Als Überprüfung des Layers und der Korrektheit der Bounding-Box (um Fehlern vorzubeugen) nutzte ich Code aus dem "bounding_box_aller_layer.py"-Skript, um uns die räumliche Ausdehnung unseres Shapefiles ausgeben zu lassen. 

In [ ]:
"""
BOUNDING BOX VON EINEM LAYER 
"""
from qgis.core import QgsProject 

layer_name = "landflaeche_uruguay"
einzelner_layer = QgsProject.instance().mapLayersByName(layer_name)[0]

if einzelner_layer and isinstance(einzelner_layer, QgsVectorLayer):
    
     extent = einzelner_layer.extent()

     xmin = extent.xMinimum()
     xmax = extent.xMaximum()
     ymin = extent.yMinimum()
     ymax = extent.yMaximum()

     print(f"Bounding-Box des Layers {einzelner_layer.name()}:")
     print(f"  Xmin: {xmin:.8f}")
     print(f"  Xmax: {xmax:.8f}")
     print(f"  Ymin: {ymin:.8f}")
     print(f"  Ymax: {ymax:.8f}")

     bbox_coords = (xmin, xmax, ymin, ymax)

Unser Ergebnis in der QGIS-Python-Konsole: 

Bounding-Box des Layers landflaeche_uruguay:
  Xmin: -58.44272232
  Xmax: -53.09424591
  Ymin: -34.97402954
  Ymax: -30.079689038913

Das Ergebnis war korrekt und stimmte mit der Ausgabe des Plugins überein. In unserem Fall wurden nach Abzug der Duplikate 14 Bilder gefunden. Der anschließende "Running"-Prozess kann etwas dauern.
Bei "Download Images" loggen wir uns mit den Copernicus-Logindaten ein, wählen das im vorherigen Schritt erstellte GPKG als Input aus, erneut unseren Output-Ordner sowie bei "OPTIONS" die gewünschten Bänder. Für diese Auswertung werden das True Color Image (TCI) sowie die Bänder B02, B03, B04, B05, B06, B08 und B11 heruntergeladen. 
Es kann auch ein kleineres Untersuchungsgebiet gewählt werden, in meinem Fall werden nun 89 Bilder à ca. 400 MB Daten heruntergeladen. Das hat den Vorteil eines sehr großen Datensatzes für Machine Learning Methoden, Datenvalidierung und mehr, ist aber speicherintensiv. 